# Term Frecuency x Inverse Document Frecuency (TF IDF)


In [1]:
import io
import re
from functools import reduce
from collections import Counter
from numpy import log
from pandas import DataFrame

In [2]:
with io.open('stop-words-spanish.txt', mode='r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
# stopwords

In [3]:
# AMLO_28042013
with io.open('Data/AMLO_20180401.txt', mode='r', encoding='utf-8') as f:
    boletin = f.read().splitlines()
boletin

['Andrés Manuel López Obrador, candidato a la Presidencia de México, inicia en Santa Catarina un recorrido por municipios de Nuevo Léon.',
 'Más tarde encabezará asamblea en la plaza principal de Ciudad Benito Juárez y a las 18:00 horas en la plaza central de Apodaca, N.L.',
 'Santa Catarina, Nuevo León, 02 de abril de 2018.-  Andrés Manuel López Obrador aseguró que en campaña todo será amor y paz, “que se ahorren cualquier provocación los candidatos de los otros partidos, porque no voy a contestarles”, “no me voy a enganchar en ningún debate, no me voy a enojar”.',
 'Informó que sus asesores de los pueblos de México me están recomendando que yo no me enoje, que no conteste a ninguna provocación y recalcó que no va a polemizar, no va a testerear, ni caerá en ninguna provocación de sus adversarios.',
 'En otro tema, López Obrador adelantó que no pasará nada el primero de julio cuando se lleven a cabo las elecciones presidenciales, “lo único que va a suceder es de que se va a iniciar una

### Obtener palabras

Obtenemos las palabras en minúsculas


In [4]:
documentos = [re.findall('[a-zA-zñáéíóúüÁÉÍÓÚÜÑ]+', parrafo.lower()) for (index, parrafo) in enumerate(boletin)]
# documentos

Obtenemos documentos sin stopwords

In [5]:
documentos = [[palabra for palabra in documento if not (palabra in stopwords)] for documento in documentos]
total_palabras = len(documentos)
# documentos

Contar las palabras por documento y el total de palabras

In [6]:
cuenta_por_documento = [Counter(parrafo) for parrafo in documentos]
cuenta_total_palabras = reduce((lambda x, y: x + y), cuenta_por_documento)
palabras = [palabra for (palabra, cuenta) in cuenta_total_palabras.items()]
# cuenta_por_documento

Definimos función para suavizar la cuenta de palabras.

In [7]:
def bm25(contador, k=25):
    return {
        key: ((cuenta + 1)*k)/(cuenta+k) for (key, cuenta) in contador.items()
    }

Calculamos tf para cada uno de los documentos 

In [8]:
tfs = [bm25(c_doc) for c_doc in cuenta_por_documento]
# tfs

Definimos la función IDF

In [9]:
def idf(palabra, documentos):
    return log(
        len(documentos) / 
        (1+reduce(lambda cuenta, documento : cuenta + (1 if (palabra in documento) else 0), documentos, 0))
    )
# documento for documento in documentos if (palabra in documento)
# ]))

Calculamos el idf de todas las palabras

In [10]:
idfs = {
    palabra: idf(palabra, documentos) for palabra in palabras
}
# idfs

Calculamos tfxIDF

In [11]:
tfidfs = [
    {
        palabra: idfs[palabra]*tf
        for (palabra, tf) in tf_n.items()
    }
    for tf_n in tfs
]
# tfidfs

In [12]:
def obtener_keywords(num_keywords=5):
    keywords = []
    for tfidf_n in tfidfs:
        keywords_n = sorted(tfidf_n.items() ,  key=lambda x: x[1], reverse=True)[:num_keywords]
#         print([keyword[0] for keyword in keywords_n])
        keywords.append([keyword[0] for keyword in keywords_n])
    return keywords

In [13]:
keywords = obtener_keywords(6)
# keywords

### Palabras clave por párrafo

In [14]:
keywords_dict = {'Párrafo {0}'.format(n+1):kw_n for (n, kw_n) in enumerate(keywords)}
df = DataFrame(keywords_dict).transpose()
df

,0,1,2,3,4,5
Párrafo 1,inicia,recorrido,municipios,léon,presidencia,andrés
Párrafo 2,plaza,encabezará,asamblea,principal,horas,central
Párrafo 3,voy,abril,campaña,amor,paz,ahorren
Párrafo 4,ninguna,provocación,informó,asesores,recomendando,enoje
Párrafo 5,pasará,lleven,presidenciales,único,suceder,iniciar
Párrafo 6,dosis,entiende,nerviosos,salió,mayor,sino
Párrafo 7,situación,violencia,prevalece,norte,cómo,pidiendo
Párrafo 8,donald,tlc,trump,países,impulsará,acuerdos
Párrafo 9,trabajador,gana,dólares,hora,falta,logré
Párrafo 10,resuelven,negociación,incluir,migratorio,momento,convencerá


### Palabras más usadas

In [15]:
DataFrame(sorted(cuenta_total_palabras.items(),  key=lambda x: x[1], reverse=True)[:15], columns=['Palabra', 'Cuenta'])

,Palabra,Cuenta
0,va,14
1,nuevo,13
2,lópez,11
3,obrador,11
4,méxico,9
5,aeropuerto,9
6,técnicos,8
7,santa,7
8,león,7
9,catarina,6
